# Import Libs

In [1]:
from __future__ import division
from __future__ import print_function
%matplotlib inline

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.multilabel import multilabel_sample_dataframe, multilabel_train_test_split
from features.SparseInteractions import SparseInteractions
from models.metrics import multi_multi_log_loss


# Load Data and Display Shape

In [2]:
path_to_training_data = os.path.join(os.pardir, 'data', 'TrainingSet.csv')
df = pd.read_csv(path_to_training_data, index_col=0)
print(df.shape)

(400277, 25)


# Set Display Max Columns to 500

In [3]:
pd.set_option('display.max_columns', 500)

# Display First 5 Rows of DF

In [4]:
df.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


# Display Last 5 Rows of DF

In [5]:
df.tail()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
109283,Professional Development,ISPD,Shared Services,Non-School,Unspecified,Instructional Coach,Other Compensation/Stipend,NO_LABEL,PreK-12 Operating,WORKSHOP PARTICIPANT,NaN,NaN,CURRICULUM RESOURCE TEACHER,NaN,NaN,NaN,STAFF DEV AND INSTR MEDIA,NaN,INST STAFF TRAINING SVCS,NaN,NaN,48.620000,NaN,GENERAL FUND,STAFF DEV AND INSTR MEDIA
102430,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Base Salary/Compensation,NO_LABEL,PreK-12 Operating,SALARIES OF PART TIME EMPLOYEE,NaN,FEDERAL GDPG FUND - FY,"Teacher,Retrd Shrt Term Sub",Regular,NaN,NaN,NaN,0.00431,"TITLE II,D",NaN,PROFESSIONAL-INSTRUCTIONAL,128.824985,INSTRUCTIONAL STAFF TRAINING,NaN,INSTRUCTIONAL STAFF
413949,Parent & Community Relations,NO_LABEL,School Reported,School,NO_LABEL,Other,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,NaN,NaN,School Liaison,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,PARENT/TITLE I,4902.290000,Misc,Schoolwide Schools,NaN
433672,Library & Media,Instruction,School on Central Budgets,Non-School,Unspecified,Librarian,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,EDUCATIONAL RESOURCE SERVICES,LEVY OVERRIDE,Library Technician II,NaN,NaN,NaN,ED RESOURCE SERVICES,NaN,NON-PROJECT,NaN,OFFICE/ADMINISTRATIVE SUPPORT,4020.290000,MEDIA SUPPORT SERVICES,NaN,INSTRUCTIONAL STAFF
415831,Substitute Compensation,Instruction,School Reported,School,Poverty,Substitute,Substitute Compensation,Non PreK,PreK-12 Operating,Salaries And Wages For Substitute Professionals,NaN,"""Title Part A Improving Basic Programs""",TEACHER SUBSTITUTE POOL,NaN,Multilingual Dist Prof Development,Inservice Substitute Teachers Grant Funded,School,NaN,Instruction,Instruction And Curriculum,CERTIFIED SUBSTITUTE,46.530000,Accelerated Education,"""Title Part A Improving Basic Programs""",MISCELLANEOUS


# Separate Training and Test Data

In [38]:
LABELS = ['Function',
          'Use',
          'Sharing',
          'Reporting',
          'Student_Type',
          'Position_Type',
          'Object_Type', 
          'Pre_K',
          'Operating_Status']

NON_LABELS = [c for c in df.columns if c not in LABELS]

SAMPLE_SIZE = 40000

sampling = multilabel_sample_dataframe(df,
                                       pd.get_dummies(df[LABELS]),
                                       size=SAMPLE_SIZE,
                                       min_count=25,
                                       seed=43
                                      )

dummy_labels = pd.get_dummies(sampling[LABELS])

X_train, X_test, y_train, y_test = multilabel_train_test_split(sampling[NON_LABELS], 
                                                               dummy_labels,
                                                               0.1,
                                                               min_count=3,
                                                               seed=43)

# Combine Text Columns

In [39]:
NUMERIC_COLUMNS = ['FTE', "Total"]

def combine_text_columns(df, to_drop=NUMERIC_COLUMNS + LABELS):
    '''
        Takes the dataset as read in,
        drops the non-feature, non-text columns
        and then combine all of the text columns
        into a single vector that has all of the text for a row
        
        :param df: The data frame
        :param to_drop (optional): Removes the numeric and label columns by default
    '''
    # drop non-text columns that are in the df
    to_drop = set(to_drop) & set(df.columns.tolist())
    text_data = df.drop(to_drop, axis=1)
    
    # replace nans with blanks
    text_data.fillna('', inplace=True)
    
    # joins all of the text items in a row (axis=1)
    return text_data.apply(lambda x: ' '.join(x), axis=1)

# Define Get Text and Numeric Data Functions

In [40]:
from sklearn.preprocessing import FunctionTransformer

get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)

# Get Text First 5 Rows

In [41]:
get_text_data.fit_transform(df.head())

134338       Teacher-Elementary        KINDERGARTEN  KIN...
206341    CONTRACTOR SERVICES BOND EXPENDITURES BUILDING...
326408    Personal Services - Teachers   TCHER 2ND GRADE...
364634    EMPLOYEE BENEFITS TEACHER SUBS GENERAL FUND Te...
47683     TEACHER COVERAGE FOR TEACHER TEACHER SUBS GENE...
dtype: object

# Get Numeric First 5 Rows

In [42]:
get_numeric_data.fit_transform(df.head())

,FTE,Total
134338,1.0,50471.810
206341,NaN,3477.860
326408,1.0,62237.130
364634,NaN,22.300
47683,NaN,54.166


# Make Multi-Multi Log Loss Scorer

In [43]:
from sklearn.metrics.scorer import make_scorer

log_loss_scorer = make_scorer(multi_multi_log_loss)

# Import Sklearn Libs

In [44]:
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler

# Alphanumeric Tokens

In [45]:
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Number of Features Before Adding Interactions 

In [46]:
chi_k = 300

# Pipeline

In [47]:
pl = Pipeline([
    ('union', 
         FeatureUnion(
             transformer_list = [
                 ('numeric_features', 
                      Pipeline([
                          ('selector', get_numeric_data),
                          ('imputer', SimpleImputer())
                      ])
                 ),
                 ('text_features', Pipeline([
                     ('selector', get_text_data),
                     ('vectorizer', 
                          CountVectorizer(
                              token_pattern=TOKENS_ALPHANUMERIC,
                              binary=False,
                              ngram_range=(1, 2)
                          )
                     ),
                     ('dim_red', SelectKBest(chi2, chi_k))
                 ])
                 )
             ]
         )
    ),
    ('interaction', SparseInteractions(degree=2)),
    ('scale', MaxAbsScaler()),
    ('classifier', OneVsRestClassifier(LogisticRegression()))
])

# Fit the Pipeline to Training Data

In [48]:
%%time
pl.fit(X_train, y_train.values)

CPU times: user 11min 54s, sys: 8.8 s, total: 12min 3s
Wall time: 6min 14s


Pipeline(memory=None,
         steps=[('union',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('numeric_features',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FunctionTransformer(accept_sparse=False,
                                                                                      check_inverse=True,
                                                                                      func=<function <lambda> at 0xa1e166158>,
                                                                                      inv_kw_args=None,
                                                                                      inverse_func=None,
                                                                                      kw_args=None,
                                          

# Log Loss Score of the Trained Pipeline

In [49]:
print('Log loss score of the trained pipeline on the test data', 
          log_loss_scorer(pl, X_test, y_test.values)
     )

Log loss score of the trained pipeline on the test data 2.210502021157787


In [31]:
class_column_indices = [list(range(37)),
                            list(range(37, 48)),
                            list(range(48, 51)),
                            list(range(51, 76)),
                            list(range(76, 79)),
                            list(range(79, 82)),
                            list(range(82, 87)),
                            list(range(87, 96)),
                            list(range(96, 104))]

In [25]:
y_pred = pl.predict_proba(X_test)

In [27]:
y_pred.shape

(8005, 104)

In [34]:
predicted = y_pred
actual = y_test.values

In [35]:
actual.shape

(8005, 104)

In [32]:
class_scores = np.ones(len(class_column_indices), dtype=np.float64)
class_scores

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [36]:
eps = 1e-15

In [37]:
for k, this_class_indices in enumerate(class_column_indices):
    preds_k = predicted[:, this_class_indices].astype(np.float64)
    preds_k /= np.clip(preds_k.sum(1).reshape(-1, 1), eps, np.inf)
    actual_k = actual[:, this_class_indices]
    y_hats = np.clip(preds_k, eps, 1 - eps)
    sum_logs = np.sum(actual_k * np.log(y_hats))
    class_scores[k] = (-1.0 / actual.shape[0]) * sum_logs
print(class_scores)

[3.51515425e-01 1.50421116e+00 4.69062544e-02 2.44879215e+00
 1.11397205e-04 1.05694195e-03 2.42822799e-02 1.03537580e-01
 1.73130914e+00]


# Predict Holdout Set and Write Submission

In [21]:
path_to_holdout_data = os.path.join(os.pardir, 'data', 'TestSet.csv')

# Load holdout data
holdout = pd.read_csv(path_to_holdout_data, index_col=0)

# make predictions
predictions = pl.predict_proba(holdout)

# format correctly in new data frame
pred_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS]).columns,
                       index=holdout.index,
                       data=predictions)



In [22]:
print(pred_df.head())

        Function_Aides Compensation  Function_Career & Academic Counseling  \
180042                     0.000157                               0.000328   
28872                      0.000127                               0.000204   
186915                     0.026392                               0.001528   
412396                     0.026360                               0.001529   
427740                     0.001705                               0.000389   

        Function_Communications  Function_Curriculum Development  \
180042                 0.000038                         0.000609   
28872                  0.000063                         0.000133   
186915                 0.000031                         0.000642   
412396                 0.000031                         0.000641   
427740                 0.000981                         0.000041   

        Function_Data Processing & Information Services  \
180042                                         0.005928   
2887